In [1]:
import pickle
import shutil
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.ImageDataset import ImageDataset
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/Gaussian_10.pkl', 'rb') as f:
    gaussian_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/Gaussian_20.pkl', 'rb') as f:
    gaussian_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/Gaussian_30.pkl', 'rb') as f:
    gaussian_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/Gaussian_40.pkl', 'rb') as f:
    gaussian_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
gaussian_10_normalized = normalize_max_row_norm(gaussian_10)
gaussian_20_normalized = normalize_max_row_norm(gaussian_20)
gaussian_30_normalized = normalize_max_row_norm(gaussian_30)
gaussian_40_normalized = normalize_max_row_norm(gaussian_40)

gaussian_10_observations = generate_observations(img_dataset, gaussian_10_normalized, 0.05)
gaussian_20_observations = generate_observations(img_dataset, gaussian_20_normalized, 0.05)
gaussian_30_observations = generate_observations(img_dataset, gaussian_30_normalized, 0.05)
gaussian_40_observations = generate_observations(img_dataset, gaussian_40_normalized, 0.05)

gaussian_10_train, gaussian_10_test = split_dataset(gaussian_10_observations, train_ratio = 0.9, seed = 0)
gaussian_20_train, gaussian_20_test = split_dataset(gaussian_20_observations, train_ratio = 0.9, seed = 0)
gaussian_30_train, gaussian_30_test = split_dataset(gaussian_30_observations, train_ratio = 0.9, seed = 0)
gaussian_40_train, gaussian_40_test = split_dataset(gaussian_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 5)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Memory_Residual-Additive/5 Projections'

gauss_10_proj5_weights, gauss_10_proj5_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj5', display=True)

gauss_20_proj5_weights, gauss_20_proj5_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj5', display=True)

gauss_30_proj5_weights, gauss_30_proj5_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj5', display=True)

gauss_40_proj5_weights, gauss_40_proj5_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, gauss_10_proj5_weights, gauss_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, gauss_20_proj5_weights, gauss_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, gauss_30_proj5_weights, gauss_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, gauss_40_proj5_weights, gauss_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.15948997675710255
Average test loss: 0.011298070820669333
Epoch 2/300
Average training loss: 0.06529930844571856
Average test loss: 0.009895940134508742
Epoch 3/300
Average training loss: 0.05897488435109456
Average test loss: 0.0110298741352227
Epoch 4/300
Average training loss: 0.055486474553743996
Average test loss: 0.00933764979408847
Epoch 5/300
Average training loss: 0.05320913322104348
Average test loss: 0.008755339318679438
Epoch 6/300
Average training loss: 0.05088126142157449
Average test loss: 0.00908469972676701
Epoch 7/300
Average training loss: 0.049470247397820154
Average test loss: 0.008909200750291347
Epoch 8/300
Average training loss: 0.048239920841323
Average test loss: 0.008777254226307074
Epoch 9/300
Average training loss: 0.04738866111636162
Average test loss: 0.00834872154560354
Epoch 10/300
Average training loss: 0.046577177478207485
Average test loss: 0.008327506497502328
Epoch 11/300

In [8]:
gauss_10_proj5_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj5_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj5_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj5_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj5_psnr = average_PSNR(gauss_10_proj5_model, gauss_10_proj5_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj5_psnr = average_PSNR(gauss_20_proj5_model, gauss_20_proj5_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj5_psnr = average_PSNR(gauss_30_proj5_model, gauss_30_proj5_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj5_psnr = average_PSNR(gauss_40_proj5_model, gauss_40_proj5_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj5_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 21.85
Average PSNR for Projection Layer 1 across 2500 images: 23.38
Average PSNR for Projection Layer 2 across 2500 images: 23.99
Average PSNR for Projection Layer 3 across 2500 images: 24.33
Average PSNR for Projection Layer 4 across 2500 images: 24.51
Average PSNR for Projection Layer 0 across 2500 images: 22.79
Average PSNR for Projection Layer 1 across 2500 images: 24.92
Average PSNR for Projection Layer 2 across 2500 images: 25.77
Average PSNR for Projection Layer 3 across 2500 images: 26.14
Average PSNR for Projection Layer 4 across 2500 images: 26.36
Average PSNR for Projection Layer 0 across 2500 images: 23.48
Average PSNR for Projection Layer 1 across 2500 images: 26.07
Average PSNR for Projection Layer 2 across 2500 images: 26.97
Average PSNR for Projection Layer 3 across 2500 images: 27.39
Average PSNR for Projection Layer 4 across 2500 images: 27.60
Average PSNR for Projection Layer 0 across 2500 images: 23.92
Average 

In [9]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Memory_Residual-Additive/15 Projections'

gauss_10_proj15_weights, gauss_10_proj15_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj15', display=True)

gauss_20_proj15_weights, gauss_20_proj15_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj15', display=True)

gauss_30_proj15_weights, gauss_30_proj15_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj15', display=True)

gauss_40_proj15_weights, gauss_40_proj15_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, gauss_10_proj15_weights, gauss_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, gauss_20_proj15_weights, gauss_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, gauss_30_proj15_weights, gauss_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, gauss_40_proj15_weights, gauss_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 1.841318406952752
Average test loss: 0.013515510789222187
Epoch 2/300
Average training loss: 0.6282312652269999
Average test loss: 0.011755104781024987
Epoch 3/300
Average training loss: 0.42737956953048706
Average test loss: 0.009394768654472299
Epoch 4/300
Average training loss: 0.3385455581479602
Average test loss: 0.008911108407295412
Epoch 5/300
Average training loss: 0.28525476124551563
Average test loss: 0.008993442826800877
Epoch 6/300
Average training loss: 0.25116359305381775
Average test loss: 0.008727741523749298
Epoch 7/300
Average training loss: 0.22738877329561447
Average test loss: 0.00919193244063192
Epoch 8/300
Average training loss: 0.20662408632702298
Average test loss: 0.0083265376297964
Epoch 9/300
Average training loss: 0.19541678338580662
Average test loss: 0.007830656131936444
Epoch 10/300
Average training loss: 0.18414813339710234
Average test loss: 0.007729164155821006
Epoch 11/300
Av

In [10]:
gauss_10_proj15_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj15_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj15_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj15_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj15_psnr = average_PSNR(gauss_10_proj15_model, gauss_10_proj15_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj15_psnr = average_PSNR(gauss_20_proj15_model, gauss_20_proj15_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj15_psnr = average_PSNR(gauss_30_proj15_model, gauss_30_proj15_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj15_psnr = average_PSNR(gauss_40_proj15_model, gauss_40_proj15_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj15_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 21.13
Average PSNR for Projection Layer 1 across 2500 images: 22.69
Average PSNR for Projection Layer 2 across 2500 images: 23.35
Average PSNR for Projection Layer 3 across 2500 images: 23.73
Average PSNR for Projection Layer 4 across 2500 images: 23.95
Average PSNR for Projection Layer 5 across 2500 images: 24.10
Average PSNR for Projection Layer 6 across 2500 images: 24.17
Average PSNR for Projection Layer 7 across 2500 images: 24.33
Average PSNR for Projection Layer 8 across 2500 images: 24.40
Average PSNR for Projection Layer 9 across 2500 images: 24.42
Average PSNR for Projection Layer 10 across 2500 images: 24.53
Average PSNR for Projection Layer 11 across 2500 images: 24.60
Average PSNR for Projection Layer 12 across 2500 images: 24.66
Average PSNR for Projection Layer 13 across 2500 images: 24.68
Average PSNR for Projection Layer 14 across 2500 images: 24.69
Average PSNR for Projection Layer 0 across 2500 images: 22.32
Ave

In [11]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Memory_Residual-Additive/30 Projections'

gauss_10_proj30_weights, gauss_10_proj30_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj30', display=True)

gauss_20_proj30_weights, gauss_20_proj30_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj30', display=True)

gauss_30_proj30_weights, gauss_30_proj30_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj30', display=True)

gauss_40_proj30_weights, gauss_40_proj30_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, gauss_10_proj30_weights, gauss_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, gauss_20_proj30_weights, gauss_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, gauss_30_proj30_weights, gauss_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, gauss_40_proj30_weights, gauss_40_proj30_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 15.636712245517307
Average test loss: 0.013690913281506963
Epoch 2/300
Average training loss: 7.763323093414306
Average test loss: 0.01102022772282362
Epoch 3/300
Average training loss: 5.836600591023763
Average test loss: 0.010364015301068624
Epoch 4/300
Average training loss: 4.832971425798204
Average test loss: 0.010328171860012744
Epoch 5/300
Average training loss: 3.9838033746083577
Average test loss: 0.009303914826777246
Epoch 6/300
Average training loss: 3.234006671481662
Average test loss: 0.008704828222592671
Epoch 7/300
Average training loss: 2.835748350567288
Average test loss: 0.009614075902435515
Epoch 8/300
Average training loss: 2.321161980946859
Average test loss: 0.008207414374997219
Epoch 9/300
Average training loss: 1.9815482142766316
Average test loss: 0.00884528438333008
Epoch 10/300
Average training loss: 1.6794004350238376
Average test loss: 0.008295783472557862
Epoch 11/300
Average train

In [12]:
gauss_10_proj30_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj30_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj30_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj30_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj30_psnr = average_PSNR(gauss_10_proj30_model, gauss_10_proj30_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj30_psnr = average_PSNR(gauss_20_proj30_model, gauss_20_proj30_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj30_psnr = average_PSNR(gauss_30_proj30_model, gauss_30_proj30_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj30_psnr = average_PSNR(gauss_40_proj30_model, gauss_40_proj30_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj30_psnr, f)

Average PSNR for Projection Layer 0 across 2500 images: 21.05
Average PSNR for Projection Layer 1 across 2500 images: 22.52
Average PSNR for Projection Layer 2 across 2500 images: 23.08
Average PSNR for Projection Layer 3 across 2500 images: 23.44
Average PSNR for Projection Layer 4 across 2500 images: 23.70
Average PSNR for Projection Layer 5 across 2500 images: 23.86
Average PSNR for Projection Layer 6 across 2500 images: 23.96
Average PSNR for Projection Layer 7 across 2500 images: 24.07
Average PSNR for Projection Layer 8 across 2500 images: 24.15
Average PSNR for Projection Layer 9 across 2500 images: 24.20
Average PSNR for Projection Layer 10 across 2500 images: 24.28
Average PSNR for Projection Layer 11 across 2500 images: 24.35
Average PSNR for Projection Layer 12 across 2500 images: 24.40
Average PSNR for Projection Layer 13 across 2500 images: 24.40
Average PSNR for Projection Layer 14 across 2500 images: 24.46
Average PSNR for Projection Layer 15 across 2500 images: 24.52
Av